<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/spell/Q_spell_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본		    G_inc_again_so_model.pt
 산업분류자동화    'My Drive'
'Colab Notebooks'   Screenshot_20220401-001411_Chrome.jpg


In [19]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/산업분류자동화/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [20]:
df=df[(df['digit_1'] =='Q')] #조건에 맞는 데이터 출력

In [21]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
1,id_0000072,Q,86,863,보건소,내원 환자를 대상으로,진료서비스
2,id_0000101,Q,86,862,일반의원에서,외래환자를 위주로,외과진료
3,id_0000128,Q,87,872,복지관에서,일반인들을 대상으로,비거주복지서비스업
4,id_0000254,Q,87,872,어린이집에서,보육료,취학전 아동보육
...,...,...,...,...,...,...,...
36082,id_0999859,Q,87,872,복지관에서,일반인 대상으로,사회복지 서비스
36083,id_0999871,Q,87,872,어린이집,보호자의 위탁을 받아,취학전 아동보육
36084,id_0999916,Q,87,872,NaN,재가 요양서비스 기관,NaN
36085,id_0999960,Q,87,872,어린이집에서,보호자로 위탁받아,영유아보육


In [22]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000005,Q,87,872,어린이집 보호자의 위탁을 받아 취학전아동보육
1,id_0000072,Q,86,863,보건소 내원 환자를 대상으로 진료서비스
2,id_0000101,Q,86,862,일반의원에서 외래환자를 위주로 외과진료
3,id_0000128,Q,87,872,복지관에서 일반인들을 대상으로 비거주복지서비스업
4,id_0000254,Q,87,872,어린이집에서 보육료 취학전 아동보육
...,...,...,...,...,...
36082,id_0999859,Q,87,872,복지관에서 일반인 대상으로 사회복지 서비스
36083,id_0999871,Q,87,872,어린이집 보호자의 위탁을 받아 취학전 아동보육
36084,id_0999916,Q,87,872,재가 요양서비스 기관
36085,id_0999960,Q,87,872,어린이집에서 보호자로 위탁받아 영유아보육


In [23]:
df['digit_3'].value_counts()

872    15743
862    15287
871     2128
869     1198
861      878
863      853
Name: digit_3, dtype: int64

## 맞춤법 교정

In [24]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-i07xs6hx
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-i07xs6hx


In [25]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [26]:
df.to_csv('df_Q_spellcheck.csv', index=False, encoding='utf-8-sig')